In [1]:
import os
import glob
import matplotlib.pyplot as plt
from utilities.settings import Paths, Subjects, Extensions, Params
import pandas as pd
from nilearn.masking import compute_epi_mask
import numpy as np

from sklearn.metrics import r2_score
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import Ridge, RidgeCV
from nilearn.image import math_img, mean_img
from joblib import Parallel, delayed
from nilearn.input_data import MultiNiftiMasker
from nilearn.plotting import plot_glass_brain
from sklearn.model_selection import LeaveOneOut, LeaveOneGroupOut
from nilearn.image import math_img, mean_img
import nibabel as nib
from utilities.splitter import Splitter
import sklearn
from tqdm import tqdm
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler
import pickle
from os.path import join
from tqdm import tqdm
import csv
from sklearn.metrics import r2_score

paths = Paths()
subjects_list = Subjects()
params = Params()
extensions= Extensions()

In [2]:
source = 'fMRI'
input_data_type = 'design-matrices'
output_data_type = 'ridge-indiv'
alphas = np.logspace(-3, 1, 30)
model = Ridge(alpha=0.1)
model_name = 'lstm_wikikristina_embedding-size_200_nhid_100_nlayers_3_dropout_01'
subjects = ['sub-057']

In [26]:

#########################################
############ Data retrieving ############
#########################################

def get_data(language, data_type, subject=None, source='', model=''):
    # General function for data retrieving
    # Output: list of path to the different data files
    extension = extensions.get_extension(data_type)
    sub_dir = os.listdir(paths.path2data)
    if data_type in sub_dir:
        base_path = paths.path2data
        if data_type in ['fMRI', 'MEG']:
            file_pattern = '{2}/func/{0}_{1}_{2}_run*'.format(data_type, language, subject) + extension
        else:
            file_pattern = '{}_{}_{}_run*'.format(data_type, language, model) + extension
    else:
        base_path = join(paths.path2derivatives, source)
        file_pattern = '{}_{}_{}_run*'.format(data_type, language, model) + extension
    data = sorted(glob.glob(join(base_path, '{0}/{1}/{2}'.format(data_type, language, model), file_pattern)))
    return data


def get_output_parent_folder(source, output_data_type, language, model):
    return join(paths.path2derivatives, '{0}/{1}/{2}/{3}'.format(source, output_data_type, language, model))


def get_path2output(output_parent_folder, output_data_type, language, model, run_name, extension):
    return join(output_parent_folder, '{0}_{1}_{2}_{3}'.format(output_data_type, language, model, run_name) + extension)



#########################################
###### Computation functionalities ######
#########################################

def compute(path, overwrite=False):
    # Tell us if we can compute or not
    result = True
    if os.path.isfile(path):
        result = overwrite
    return result


def check_folder(path):
    # Create adequate folders if necessary
    if not os.path.isdir(path):
        os.mkdir(path)



#########################################
################## Log ##################
#########################################

def log(subject, voxel, alpha, r2):
    """ log stats per fold to a csv file """
    logcsvwriter = csv.writer(open("test.log", "a+"))
    if voxel == 'whole brain':
        logcsvwriter.writerow([subject, voxel, np.mean(r2), np.std(r2),
                            np.min(r2), np.max(r2)])
    else:
        logcsvwriter.writerow([subject, voxel, alpha, r2])


#########################################
########## Classical functions ##########
#########################################
def get_r2_score(model, y_true, y2predict, r2_min=0., r2_max=0.99):
    # return the R2_score for each voxel (=list)
    r2 = r2_score(y_true,
                    model.predict(y2predict),
                    multioutput='raw_values')
    # remove values with are too low and values too good to be true (e.g. voxels without variation)
    return np.array([0 if (x < r2_min or x >= r2_max) else x for x in r2])



In [27]:

#########################################
################ Figures ################
#########################################

def create_maps(masker, distribution, distribution_name, subject, output_parent_folder):
    model = os.path.basename(output_parent_folder)
    language = os.path.basename(os.path.dirname(output_parent_folder))
    data_type = os.path.basename(os.path.dirname(os.path.dirname(output_parent_folder)))

    img = masker.inverse_transform(distribution)
    
    path2output_raw = join(output_parent_folder, "{0}_{1}_{2}_{3}_{4}".format(data_type, language, model, distribution_name, subject)+'.nii.gz')
    path2output_png = join(output_parent_folder, "{0}_{1}_{2}_{3}_{4}".format(data_type, language, model, distribution_name, subject)+'.png')

    nib.save(img, path2output_raw)

    display = plot_glass_brain(img, display_mode='lzry', threshold=0, colorbar=True)
    display.savefig(path2output_png)
    display.close()



#########################################
######### First level analysis ##########
#########################################



def do_single_subject(subject, fmri_filenames, design_matrices, masker, output_parent_folder, model, voxel_wised=False, alpha_list=np.logspace(-3, -1, 30)):
    # Compute r2 maps for all subject for a given model
    #   - subject : e.g. : 'sub-060'
    #   - fmri_runs: list of fMRI data runs (1 for each run)
    #   - matrices: list of design matrices (1 for each run)
    #   - masker: MultiNiftiMasker object
    fmri_runs = [masker.transform(f) for f in fmri_filenames] # return a list of 2D matrices with the values of the voxels in the mask: 1 voxel per column
    
    # compute r2 maps and save them under .nii.gz and .png formats
    if voxel_wised:
        alphas, r2_test = per_voxel_analysis(model, fmri_runs, design_matrices, subject, alpha_list)
        create_maps(masker, alphas, 'alphas', subject, output_parent_folder) # alphas
    else:
        r2_test = whole_brain_analysis(model, fmri_runs, design_matrices, subject)
    create_maps(masker, r2_test, 'r2_test', subject, output_parent_folder) # r2 test


def whole_brain_analysis(model, fmri_runs, design_matrices, subject):
    #   - fmri_runs: list of fMRI data runs (1 for each run)
    #   - matrices: list of design matrices (1 for each run)
    scores = None  # array to contain the r2 values (1 row per fold, 1 column per voxel)
    nb_runs = len(fmri_runs)
    
    for index in range(len(design_matrices)):
        scaler = StandardScaler()
        scaler.fit(design_matrices[index])
        design_matrices[index] = scaler.transform(design_matrices[index])
        const = np.ones((design_matrices[index].shape[0], 1))
        design_matrices[index] = np.hstack((design_matrices[index], const))

    logo = LeaveOneGroupOut() # leave on run out !
    for train, test in tqdm(logo.split(fmri_runs, groups=range(1, nb_runs+1))):
        fmri_data_train = np.vstack([fmri_runs[i] for i in train]) # fmri_runs liste 2D colonne = voxels et chaque row = un t_i
        predictors_train = np.vstack([design_matrices[i] for i in train])

        if type(model) == sklearn.linear_model.RidgeCV or type(model) == sklearn.linear_model.Ridge:
            print()
            #nb_samples = np.cumsum([0] + [[fmri_runs[i] for i in train][i].shape[0] for i in range(len([fmri_runs[i] for i in train]))]) # list of cumulative lenght
            #indexes = {'run{}'.format(run+1): [nb_samples[i], nb_samples[i+1]] for i, run in enumerate(train)}
            #model.cv = Splitter(indexes_dict=indexes, n_splits=nb_runs-1) # adequate splitter for cross-validate alpha taking into account groups
        print('Fitting model...')
        print(predictors_train)
        model_fitted = model.fit(predictors_train, fmri_data_train)
        pickle.dump(model_fitted, open(join(paths.path2derivatives, 'fMRI/glm-indiv/english', str(model).split('(')[0]+ '{}.sav'.format(test[0])), 'wb'))
        print('Model fitted.')

        # return the R2_score for each voxel (=list)
        r2 = get_r2_score(model_fitted, fmri_runs[test[0]], design_matrices[test[0]])
        print(r2)
        print(scores)

        # log the results
        log(subject, voxel='whole brain', alpha=None, r2=r2)

        scores = r2 if scores is None else np.vstack([scores, r2])
    result = pd.DataFrame(scores, columns=['voxel #{}'.format(i) for i in range(scores.shape[1])])
    result.to_csv(join(paths.path2derivatives, 'fMRI/glm-indiv/english', str(model).split('(')[0]+ '.csv'))
    return np.mean(scores, axis=0) # compute mean vertically (in time)


def per_voxel_analysis(model, fmri_runs, design_matrices, subject, alpha_list):
    # compute alphas and test score with cross validation
    #   - fmri_runs: list of fMRI data runs (1 for each run)
    #   - design_matrices: list of design matrices (1 for each run)
    #   - nb_voxels: number of voxels
    #   - indexes: dict specifying row indexes for each run
    nb_voxels = fmri_runs[0].shape[1]
    nb_alphas = len(alpha_list)
    nb_runs_test = len(fmri_runs)
    nb_runs_valid = nb_runs_test -1 
    alphas_cv2 = np.zeros((nb_runs_test, nb_voxels))
    scores_cv2 = np.zeros((nb_runs_test, nb_voxels))
    
    # loop for r2 computation
    cv3 = 0
    logo = LeaveOneOut() # leave on run out !
    for train_, test in logo.split(fmri_runs):
        fmri_data_train_ = [fmri_runs[i] for i in train_] # fmri_runs liste 2D colonne = voxels et chaque row = un t_i
        predictors_train_ = [design_matrices[i] for i in train_]
        
        cv2 = 0
        logo2 = LeaveOneOut() # leave on run out !
        for train, valid in logo2.split(fmri_data_train_):
            fmri_data_train = [fmri_data_train_[i] for i in train] # fmri_runs liste 2D colonne = voxels et chaque row = un t_i
            predictors_train = [predictors_train_[i] for i in train]
            dm = np.vstack(predictors_train)
            fmri = np.vstack(fmri_data_train)
            scores_cv1 = np.zeros((nb_voxels, nb_runs_valid, nb_alphas))
            
            cv1 = 0
            for alpha_tmp in tqdm(alpha_list): # compute the r2 for a given alpha for all the voxel
                model.set_params(alpha=alpha_tmp)
                model_fitted = model.fit(dm,fmri)
                r2 = get_r2_score(model_fitted, fmri_data_train_[valid[0]], predictors_train_[valid[0]])
                scores_cv1[:, cv2, cv1] = r2
                cv1 += 1
            #best_alphas_indexes = np.argmax(scores_cv1, axis=1)
            cv2 += 1
        best_alphas_indexes = np.argmax(np.mean(scores_cv1, axis=1), axis=1)
        alphas_cv2[cv3, :] = np.array([alpha_list[i] for i in best_alphas_indexes])
        fmri2 = np.vstack(fmri_data_train_)
        dm2 = np.vstack(predictors_train_)
        for voxel in tqdm(range(nb_voxels)): # loop through the voxels and fit the model with the best alpha for this voxel
            y = fmri2[:,voxel].reshape((fmri2.shape[0],1))
            model.set_params(alpha=alphas_cv2[cv3, voxel])
            model_fitted = model.fit(dm2, y)
            scores_cv2[cv3, voxel] = get_r2_score(model_fitted, 
                                            fmri_runs[test[0]][:,voxel].reshape((fmri_runs[test[0]].shape[0],1)), 
                                            design_matrices[test[0]])
            # log the results
            log(subject, voxel=voxel, alpha=alphas_cv2[cv3, voxel], r2=scores_cv2[cv3, voxel])
        cv3 += 1
        
    return np.mean(alphas_cv2, axis=0), np.mean(scores_cv2, axis=0) # compute mean vertically (in time)
    
    

In [6]:
language = 'english'

In [7]:
dm = get_data(language, input_data_type, model=model_name, source='fMRI')
fmri_runs = {subject: get_data(language, data_type=source, subject=subject) for subject in subjects}

In [8]:
output_parent_folder = get_output_parent_folder(source, output_data_type, language, model_name)
check_folder(output_parent_folder) # check if the output_parent_folder exists and create it if not

In [9]:
output_parent_folder

'/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/ridge-indiv/english/lstm_wikikristina_embedding-size_200_nhid_100_nlayers_3_dropout_01'

In [11]:

def transform_design_matrices(path):
    # Read design matrice csv file and add a column with only 1
    dm = pd.read_csv(path, header=0)
    # add the constant
    # const = np.ones((dm.shape[0], 1))
    # dm = np.hstack((dm, const))
    return dm 


def compute_global_masker(files): # [[path, path2], [path3, path4]]
    # return a MultiNiftiMasker object

    #spm_dir = '/neurospin/unicog/protocols/IRMf/Meyniel_MarkovGuess_2014'
    #mask = join(spm_dir, 'spm12/tpm/mask_ICV.nii')
    #global_mask = math_img('img>0', img=mask)
    #masker = MultiNiftiMasker(mask_img=global_mask)
    #masker.fit()

    masks = [compute_epi_mask(f) for f in files]
    global_mask = math_img('img>0.5', img=mean_img(masks)) # take the average mask and threshold at 0.5
    masker = MultiNiftiMasker(global_mask, detrend=params.pref.detrend, standardize=params.pref.standardize) # return a object that transforms a 4D barin into a 2D matrix of voxel-time and can do the reverse action
    masker.fit()
    return masker


matrices = [transform_design_matrices(run) for run in dm] # list of design matrices (dataframes) where we added a constant column equal to 1


In [12]:
def compute_global_masker(files): # [[path, path2], [path3, path4]]
    # return a MultiNiftiMasker object

    #spm_dir = '/neurospin/unicog/protocols/IRMf/Meyniel_MarkovGuess_2014'
    #mask = join(spm_dir, 'spm12/tpm/mask_ICV.nii')
    #global_mask = math_img('img>0', img=mask)
    #masker = MultiNiftiMasker(mask_img=global_mask)
    #masker.fit()

    masks = [compute_epi_mask(f) for f in files]
    global_mask = math_img('img>0.5', img=mean_img(masks)) # take the average mask and threshold at 0.5
    masker = MultiNiftiMasker(global_mask, detrend=params.pref.detrend, standardize=params.pref.standardize) # return a object that transforms a 4D barin into a 2D matrix of voxel-time and can do the reverse action
    masker.fit()
    return masker


masker = compute_global_masker(list(fmri_runs.values()))  # return a MultiNiftiMasker object ... computation is sloow


## fMRI data analysis

In [13]:
fmri = [masker.transform(f) for f in fmri_runs['sub-057']]

/volatile/anaconda3/lib/python3.7/site-packages/nilearn/signal.py:58: RuntimeWarning: invalid value encountered in sqrt
  std = np.sqrt((signals ** 2).sum(axis=0))


In [14]:
print(np.mean(fmri[0], axis=0))
print(np.var(fmri[0], axis=0))

[ 4.3461719e-09 -7.3977398e-09  7.1863755e-09 ... -2.4306859e-08
 -3.3818237e-09  9.3000159e-09]
[1.0000001  0.99999934 1.         ... 1.0000004  0.9999988  0.9999997 ]


In [15]:
np.mean(fmri[0], axis=0).max()

1.466866e-07

## design matrices analysis

In [17]:
np.percentile(np.var(matrices[0], axis=0), 10)

0.0007430509538868077

In [19]:
np.var(matrices[0])

hrf0       0.011004
hrf1       0.013050
hrf2       0.012366
hrf3       0.011697
hrf4       0.009709
hrf5       0.012320
hrf6       0.011202
hrf7       0.012861
hrf8       0.008130
hrf9       0.016026
hrf10      0.014013
hrf11      0.019131
hrf12      0.012619
hrf13      0.017643
hrf14      0.016408
hrf15      0.015829
hrf16      0.011679
hrf17      0.014610
hrf18      0.012226
hrf19      0.011136
hrf20      0.010450
hrf21      0.008489
hrf22      0.013828
hrf23      0.016744
hrf24      0.017308
hrf25      0.013293
hrf26      0.010891
hrf27      0.008866
hrf28      0.010044
hrf29      0.014817
             ...   
hrf1771    0.001743
hrf1772    0.003098
hrf1773    0.005431
hrf1774    0.003919
hrf1775    0.009133
hrf1776    0.003599
hrf1777    0.003712
hrf1778    0.003429
hrf1779    0.004771
hrf1780    0.005945
hrf1781    0.004741
hrf1782    0.004597
hrf1783    0.006184
hrf1784    0.005405
hrf1785    0.004700
hrf1786    0.002640
hrf1787    0.002110
hrf1788    0.004695
hrf1789    0.002827


## model training

In [28]:
do_single_subject('sub-057', fmri_runs['sub-057'], matrices, masker, output_parent_folder, Ridge(alpha=50))

/volatile/anaconda3/lib/python3.7/site-packages/nilearn/signal.py:58: RuntimeWarning: invalid value encountered in sqrt
  std = np.sqrt((signals ** 2).sum(axis=0))
0it [00:00, ?it/s]


Fitting model...
[[ 0.09249381  1.03316691 -0.96822362 ... -4.83329658  0.
   1.        ]
 [ 0.08029354  0.99437744 -0.92808402 ... -4.78497582  0.
   1.        ]
 [-0.1764666   0.27685935 -0.11686628 ... -3.95763289  0.
   1.        ]
 ...
 [-0.78316756  0.3423067  -0.75517662 ...  0.63125785  0.
   1.        ]
 [-1.11991281 -0.07033856 -0.8811043  ... -0.6971583   0.
   1.        ]
 [-0.97049448 -0.07851874 -0.9385073  ... -2.89512589  0.
   1.        ]]
Model fitted.


1it [00:07,  7.54s/it]

[0 0 0 ... 0 0 0]
None

Fitting model...
[[ 0.51552478  0.93670204 -0.97307604 ... -4.89541536  0.
   1.        ]
 [ 0.56233511  0.97812661 -1.00221575 ... -4.83138309  0.
   1.        ]
 [ 0.97564125  1.06628989 -0.87785006 ... -3.39458131  0.
   1.        ]
 ...
 [-0.78316756  0.3423067  -0.75517662 ...  0.63125785  0.
   1.        ]
 [-1.11991281 -0.07033856 -0.8811043  ... -0.6971583   0.
   1.        ]
 [-0.97049448 -0.07851874 -0.9385073  ... -2.89512589  0.
   1.        ]]
Model fitted.


2it [00:15,  7.75s/it]

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]

Fitting model...
[[ 0.51552478  0.93670204 -0.97307604 ... -4.89541536  0.
   1.        ]
 [ 0.56233511  0.97812661 -1.00221575 ... -4.83138309  0.
   1.        ]
 [ 0.97564125  1.06628989 -0.87785006 ... -3.39458131  0.
   1.        ]
 ...
 [-0.78316756  0.3423067  -0.75517662 ...  0.63125785  0.
   1.        ]
 [-1.11991281 -0.07033856 -0.8811043  ... -0.6971583   0.
   1.        ]
 [-0.97049448 -0.07851874 -0.9385073  ... -2.89512589  0.
   1.        ]]
Model fitted.


3it [00:25,  8.22s/it]

[0 0 0 ... 0 0 0]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Fitting model...
[[ 0.51552478  0.93670204 -0.97307604 ... -4.89541536  0.
   1.        ]
 [ 0.56233511  0.97812661 -1.00221575 ... -4.83138309  0.
   1.        ]
 [ 0.97564125  1.06628989 -0.87785006 ... -3.39458131  0.
   1.        ]
 ...
 [-0.78316756  0.3423067  -0.75517662 ...  0.63125785  0.
   1.        ]
 [-1.11991281 -0.07033856 -0.8811043  ... -0.6971583   0.
   1.        ]
 [-0.97049448 -0.07851874 -0.9385073  ... -2.89512589  0.
   1.        ]]


KeyboardInterrupt: 

## Model analysis

In [21]:
 model0 = pickle.load(open('../derivatives/fMRI/glm-indiv/english/Ridge0.sav', 'rb')) 

In [22]:


def score(model, y_true, y2predict, r2_min=0., r2_max=0.99):
    # return the R2_score for each voxel (=list)
    r2 = r2_score(y_true,
                    model.predict(y2predict),
                    multioutput='raw_values')
    # remove values with are too low and values too good to be true (e.g. voxels without variation)
    return r2



In [23]:
for index in range(9): print(score(model0, fmri[index], matrices[index]))

[-0.93509985 -0.64446331 -0.88661538 ... -0.63107091 -0.55321624
 -0.42332446]
[0.81009184 0.78868673 0.66975513 ... 0.69780812 0.5355706  0.67318141]
[0.76560918 0.75443431 0.72637514 ... 0.83806554 0.47992103 0.65503762]
[0.85089121 0.69745401 0.71986867 ... 0.82612955 0.79959495 0.80246686]
[0.86092896 0.84941064 0.8131006  ... 0.84129548 0.86621197 0.85574448]
[0.77404616 0.78099343 0.69418221 ... 0.6157649  0.64726411 0.68747466]
[0.78109189 0.76884263 0.67936397 ... 0.69429171 0.45493624 0.40440284]
[0.79776277 0.73603875 0.62851883 ... 0.69931141 0.66010711 0.64442444]
[0.77092608 0.76708132 0.66163465 ... 0.74660716 0.66546268 0.75174625]


In [24]:
r2_score(fmri[0], (np.dot(matrices[0], model0.coef_.T) + model0.intercept_), multioutput='raw_values')

array([-0.93509985, -0.64446331, -0.88661538, ..., -0.63107091,
       -0.55321624, -0.42332446])

In [25]:
def show(data): 
    plt.figure(figsize=(7,5)) 
    plt.plot(data) 
    plt.show() 

show((np.dot(matrices[0], model0.coef_.T) + model0.intercept_ - fmri_runs[0])[:, :5])

KeyError: 0

Command line: 

python ridge-indiv.py --language english --model_name lstm_wikikristina_embedding-size_200_nhid_100_nlayers_3_dropout_01 --parallel  --subjects sub-057